In [ ]:
# Install the required pip packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install python-dotenv langchain unstructured[pdf] openai==0.28.1 chromadb toktoken

In [ ]:
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI

In [ ]:
# All Azure deployment information is stored in a .env file
import os
from io import StringIO
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
loader = UnstructuredFileLoader('Sample.pdf')
documents = loader.load()

display(documents)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=8000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

display(texts)

In [ ]:
#### https://python.langchain.com/docs/integrations/text_embedding/azureopenai

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="EMBEDDING",
    openai_api_version = "2023-09-15-preview",
)

display(embeddings)

doc_search = Chroma.from_documents(texts,embeddings)
chain = RetrievalQA.from_chain_type(llm=AzureOpenAI(model_kwargs={'engine':'DAVINCI'}),chain_type='stuff', retriever = doc_search.as_retriever())

In [ ]:
query = 'What is the main topic of the text? Use only one sentence of max 20 words'
chain.run(query)